# Proposed Architecture

This new notebook will add two major modules on top of our current pipeline:

- ACE (Adaptive Contrastive Enhancement) — a self-supervised contrastive pretraining block.

- LMAF (Lung-Masked Attention Fusion) — a lightweight attention mechanism that focuses on lung regions.

## Proposed Name: DensePneumo-ACE

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms, models
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from dataset import CXRDataset 
import os, warnings
warnings.filterwarnings("ignore")

# Paths
DATA_DIR = "../data/rsna/images"
LABELS_CSV = "../data/rsna/labels.csv"
SAVE_PATH = "model_weights/densepneumo_ace.pt"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [2]:
# Image preprocessing (same as baseline)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

df = pd.read_csv(LABELS_CSV)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

train_dataset = CXRDataset(train_df, DATA_DIR, transform)
val_dataset = CXRDataset(val_df, DATA_DIR, transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)


In [3]:
# Lung Mask Utility (Pre Computed Lung Masks)
def generate_pseudo_lung_mask(x):
    """Simple pseudo-mask based on brightness thresholds for CXR (placeholder)."""
    # x: [B, 3, H, W]
    gray = x.mean(dim=1, keepdim=True)
    mask = (gray > 0.2).float() * (gray < 0.8).float()
    return mask


In [4]:
#ACE Module (Adaptive Constrastive Enhancement)
class ContrastiveHead(nn.Module):
    def __init__(self, feature_dim=1024, proj_dim=256):
        super().__init__()
        self.fc1 = nn.Linear(feature_dim, proj_dim)
        self.fc2 = nn.Linear(proj_dim, proj_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.normalize(self.fc2(x), dim=-1)
        return x

def contrastive_loss(z_i, z_j, temperature=0.5):
    """InfoNCE loss for positive pair (z_i, z_j)."""
    batch_size = z_i.size(0)
    z = torch.cat([z_i, z_j], dim=0)
    sim = F.cosine_similarity(z.unsqueeze(1), z.unsqueeze(0), dim=2)
    sim = torch.exp(sim / temperature)
    mask = torch.eye(2*batch_size, device=z.device).bool()
    sim = sim.masked_fill(mask, 0)
    pos = torch.exp(F.cosine_similarity(z_i, z_j) / temperature)
    loss = -torch.log(pos / (sim.sum(dim=1)[:batch_size] + 1e-8)).mean()
    return loss


In [5]:
#Pretraining with ACE
backbone = models.densenet121(pretrained=True)
feature_dim = backbone.classifier.in_features
backbone.classifier = nn.Identity()

proj_head = ContrastiveHead(feature_dim, proj_dim=256).to(device)
backbone = backbone.to(device)

optimizer = torch.optim.Adam(list(backbone.parameters()) + list(proj_head.parameters()), lr=1e-4)

print("Pretraining with ACE for 1 epoch...")
backbone.train()
for imgs, _ in tqdm(train_loader):
    imgs = imgs.to(device)
    # Create augmented views (crop + jitter)
    aug1 = imgs + 0.05 * torch.randn_like(imgs)
    aug2 = torch.flip(imgs, dims=[-1])
    h_i = backbone(aug1)
    h_j = backbone(aug2)
    z_i = proj_head(h_i)
    z_j = proj_head(h_j)
    loss = contrastive_loss(z_i, z_j)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print("ACE pretraining done.")


Pretraining with ACE for 1 epoch...


100%|██████████| 1512/1512 [55:42<00:00,  2.21s/it]

ACE pretraining done.


In [6]:
torch.save(backbone.state_dict(), "model_weights/ace_pretrained_backbone.pt")

In [10]:
#LMAF Layer (Lung Masked Attention Fusion)
class LungMaskedAttentionFusion(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels//2, 1)
        self.conv2 = nn.Conv2d(in_channels//2, 1, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, features, lung_mask):
        # Downsample lung_mask to feature map resolution
        if lung_mask.shape[-1] != features.shape[-1]:
            lung_mask = F.interpolate(lung_mask, size=features.shape[2:], mode="bilinear", align_corners=False)
        
        attn_map = self.sigmoid(self.conv2(F.relu(self.conv1(features))))
        fused = features * attn_map * lung_mask
        return fused


In [11]:
#Combine into DensePneumoACE Model
class DensePneumoACE(nn.Module):
    def __init__(self, pretrained_backbone):
        super().__init__()
        self.features = pretrained_backbone.features
        self.lmaf = LungMaskedAttentionFusion(1024)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(1024, 1)

    def forward(self, x):
        lung_mask = generate_pseudo_lung_mask(x)
        features = self.features(x)
        fused = self.lmaf(features, lung_mask)
        pooled = self.pool(fused).view(fused.size(0), -1)
        return self.classifier(pooled)


In [12]:
#Training Loop
model = DensePneumoACE(backbone).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
EPOCHS = 3

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        imgs, labels = imgs.to(device), labels.to(device).float().unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{EPOCHS}]  Train Loss: {running_loss/len(train_loader):.4f}")


Epoch 1/3:   0%|          | 0/1512 [00:00<?, ?it/s]

Epoch 1/3: 100%|██████████| 1512/1512 [34:38<00:00,  1.37s/it]


Epoch [1/3]  Train Loss: 0.4312


Epoch 2/3: 100%|██████████| 1512/1512 [36:03<00:00,  1.43s/it]


Epoch [2/3]  Train Loss: 0.3859


Epoch 3/3: 100%|██████████| 1512/1512 [35:45<00:00,  1.42s/it]

Epoch [3/3]  Train Loss: 0.3552


In [13]:
#Validation Loop
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

model.eval()
y_true, y_pred, y_prob = [], [], []
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        probs = torch.sigmoid(outputs).squeeze()
        preds = (probs > 0.5).int()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_prob.extend(probs.cpu().numpy())

acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
auc = roc_auc_score(y_true, y_prob)
print(f"Val Acc: {acc:.3f}  F1: {f1:.3f}  AUC: {auc:.4f}")


Val Acc: 0.842  F1: 0.725  AUC: 0.9058


In [15]:
os.makedirs(os.path.dirname(SAVE_PATH), exist_ok=True)
torch.save(model.state_dict(), SAVE_PATH)
print(f"Model saved to {SAVE_PATH}")


Model saved to model_weights/densepneumo_ace.pt
